In [1]:
import numpy as np

import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
import sklearn

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam

from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
import art
import fairlearn

from IPython.display import Markdown, display
np.random.seed(1)

In [2]:
def run_classification_metrics(CM:ClassificationMetric):
    return np.array([
        round(CM.accuracy(), 4),
        round(CM.theil_index(), 4),
        round(CM.consistency()[0], 4),
        round(CM.false_positive_rate_difference(), 4),
        round(CM.false_negative_rate_difference(), 4),
        round(CM.error_rate_difference(), 4),
        round(CM.false_discovery_rate_difference(), 4),
        round(CM.false_omission_rate_difference(), 4)
    ])

In [3]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    return np.array([
        round(BLDM.statistical_parity_difference(), 4), # negative means privileged bias
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
    ])

In [9]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier=None, 
                 preprocessing_algo=None, inprocessing_algo=None):
    if inprocessing_algo is not None:
        inprocessing_algo.fit(dataset_train)
        results = inprocessing_algo.predict(dataset_test.copy())
        #print("inprocessing algo")
        #print(results)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = results.labels
        #dataset_test_pred = results
    else:
        classifier.fit(dataset_train.features, dataset_train.labels.ravel())
        results = classifier.predict(dataset_test.features)
        #print(results)
        if isinstance(classifier, sklearn.linear_model.LinearRegression):
            results = np.rint(results)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = np.array([results]).transpose()

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    #print(CM.binary_confusion_matrix())
    return np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

In [10]:
def get_comparison_algo(inprocessing_algo):
    if isinstance(inprocessing_algo, inprocessing.PrejudiceRemover):
        return sklearn.linear_model.LogisticRegression(solver='saga')
    if isinstance(inprocessing_algo, inprocessing.GerryFairClassifier):
        return sklearn.linear_model.LinearRegression()
    if isinstance(inprocessing_algo, inprocessing.MetaFairClassifier):
        return BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
    if isinstance(inprocessing_algo, inprocessing.ExponentiatedGradientReduction):
        return sklearn.linear_model.LogisticRegression(solver='lbfgs')
    

In [11]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "Prejudice Remover"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "Adversarial Debiasing"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ART Classifier"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "Exp Grad Reduction"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GerryFair Classifier"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GridSearch Reduction"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MetaFair Classifier"

In [12]:
def run_inproc_algo(dataset, unprivileged_groups, privileged_groups, inprocessing_algo):
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

    standard = get_comparison_algo(inprocessing_algo)

    metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, standard)
    df = pd.DataFrame(metrics, columns=[get_model_name(standard)])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "error rate difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    df[get_model_name(inprocessing_algo)] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=inprocessing_algo)
    return df

## Inprocessing Algos Analysis

In [14]:
dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

display(run_inproc_algo(dataset, unprivileged_groups, privileged_groups, inprocessing.PrejudiceRemover(sensitive_attr=dataset.protected_attribute_names[0])))
display(run_inproc_algo(dataset, unprivileged_groups, privileged_groups, inprocessing.GerryFairClassifier()))
display(run_inproc_algo(dataset, unprivileged_groups, privileged_groups, inprocessing.MetaFairClassifier()))

The max_iter was reached which means the coef_ did not converge


,Logistic Regression,Prejudice Remover
accuracy,0.7832,0.8322
theil index,0.2052,0.1308
consistency,0.8463,0.8463
false positive rate difference,-0.0161,-0.1162
false negative rate difference,-0.0029,0.2166
error rate difference,-0.1373,-0.1259
false discovery rate difference,0.2394,-0.0686
false omission rate difference,-0.1671,-0.0933
stat parity difference,-0.1981,-0.1981
priv base rate,0.3111,0.3111


,Linear Regression,GerryFair Classifier
accuracy,0.8341,0.8092
theil index,0.1480,0.2039
consistency,0.8458,0.8458
false positive rate difference,-0.0596,-0.0151
false negative rate difference,0.1147,0.0897
error rate difference,-0.1178,-0.1392
false discovery rate difference,0.0249,0.0135
false omission rate difference,-0.1165,-0.1537
stat parity difference,-0.1988,-0.1988
priv base rate,0.3109,0.3109


covariance is not positive-semidefinite.


,Meta Classifier,MetaFair Classifier
accuracy,0.8554,0.7426
theil index,0.1243,0.0790
consistency,0.8414,0.8414
false positive rate difference,-0.0731,-0.2716
false negative rate difference,0.1717,0.1192
error rate difference,-0.1002,-0.1603
false discovery rate difference,-0.0760,0.0661
false omission rate difference,-0.0881,-0.0294
stat parity difference,-0.2014,-0.2014
priv base rate,0.3115,0.3115


# Disregard everything underneath

In [64]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

'''LR = sklearn.linear_model.LogisticRegression(solver='sag').fit(dataset_train.features, dataset_train.labels.ravel())
pred = LR.predict(dataset_test.features)
print(pred)'''
LR = sklearn.linear_model.LinearRegression().fit(dataset_train.features, dataset_train.labels.ravel())
pred = np.rint(LR.predict(dataset_test.features))
print(pred)

'''PR = inprocessing.PrejudiceRemover(sensitive_attr=dataset.protected_attribute_names[0])
PR.fit(dataset_train)
print("initial test not using algo func")
print(PR.predict(dataset_test))'''

PR = inprocessing.PrejudiceRemover(sensitive_attr=dataset.protected_attribute_names[0])

GF = inprocessing.GerryFairClassifier()

MF = inprocessing.MetaFairClassifier()

EGR = inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="EqualizedOdds")

metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, LR)
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]
#BLDM = BinaryLabelDatasetMetric(dataset, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
#CM = ClassificationMetric(dataset_test, PR.predict(dataset_test), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
#df["Prejudice Remover"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=PR)
#df["GerryFair Classifier"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=GF)
df["MetaFair Classifier"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=GF)
df


[ 1.  0.  1. ... -0.  0.  0.]


KeyboardInterrupt: 

In [ ]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("German Dataset"))

dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Adult Dataset"))

dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Bank Dataset"))

dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Compas Dataset"))

In [106]:
tf.compat.v1.disable_eager_execution()
dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

sess1 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var1", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_1:
    AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_1, sess=sess1, debias=False)

    AD.fit(dataset_train)
    dataset_test_pred = AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    AD_metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess1.close()

metrics = AD_metrics
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]

sess2 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var2", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_2:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_2, sess=sess2)

    fair_AD.fit(dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w/o dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
sess2.close()

'''sess3 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var3", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_3:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)

    fair_AD.fit(dataset_train.copy())
    fair_dataset_train = fair_AD.transform(dataset_train)
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)
    fair_AD.fit(fair_dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess3.close()'''

df



epoch 0; iter: 0; batch classifier loss: 641.579346
epoch 1; iter: 0; batch classifier loss: 32.879578
epoch 2; iter: 0; batch classifier loss: 17.136475
epoch 3; iter: 0; batch classifier loss: 12.746425
epoch 4; iter: 0; batch classifier loss: 9.400681
epoch 5; iter: 0; batch classifier loss: 6.140807
epoch 6; iter: 0; batch classifier loss: 2.270022
epoch 7; iter: 0; batch classifier loss: 2.144179
epoch 8; iter: 0; batch classifier loss: 0.836270
epoch 9; iter: 0; batch classifier loss: 0.199547
epoch 10; iter: 0; batch classifier loss: 0.470885
epoch 11; iter: 0; batch classifier loss: 0.412202
epoch 12; iter: 0; batch classifier loss: 0.250243
epoch 13; iter: 0; batch classifier loss: 0.263953
epoch 14; iter: 0; batch classifier loss: 0.381343
epoch 15; iter: 0; batch classifier loss: 0.254617
epoch 16; iter: 0; batch classifier loss: 0.229809
epoch 17; iter: 0; batch classifier loss: 0.286453
epoch 18; iter: 0; batch classifier loss: 0.254371
epoch 19; iter: 0; batch classifier 

ValueError: None values not supported.

In [2]:
def print_fairness_metrics(CM:ClassificationMetric):
    print(f"accuracy = {round(CM.accuracy(), 4)}")
    print(f"theil index (goal:0) = {round(CM.theil_index(), 4)}")
    print(f"binary confusion matrix = {CM.binary_confusion_matrix()}")
    print(f"consistency (goal:1) = {round(CM.consistency()[0], 4)}")
    print(f"false positive rate difference (negative:privileged bias) = {round(CM.false_positive_rate_difference(), 4)}")
    print(f"false negative rate difference (negative:privileged bias) = {round(CM.false_negative_rate_difference(), 4)}")

In [41]:
def compare_fairness_metrics_as_df(CM1:ClassificationMetric, CM2:ClassificationMetric,
                                   intervention:str) -> pd.DataFrame:
    metrics = np.array([[round(CM1.accuracy(), 4), round(CM2.accuracy(), 4)],
        [round(CM1.theil_index(), 4), round(CM2.theil_index(), 4)],
        [round(CM1.consistency()[0], 4), round(CM2.consistency()[0], 4)],
        [round(CM1.false_positive_rate_difference(), 4), round(CM2.false_positive_rate_difference(), 4)],
        [round(CM1.false_negative_rate_difference(), 4), round(CM2.false_negative_rate_difference(), 4)]]
    )
    df = pd.DataFrame(metrics, columns=["no intervention", intervention])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference"]
    return df

In [9]:
def analyze_debiasing_algos(dataset, privileged_groups, unprivileged_groups, classifier, 
                             preprocessing_algo:preprocessing = None, inprocessing_algo:inprocessing = None, 
                             postprocessing_algo:postprocessing = None):
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)
    classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    results = classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM1 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)

    if preprocessing_algo is not None:
        dataset_train = preprocessing_algo.fit_transform(dataset_train)
    fair_classifier = clone(classifier)
    fair_classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    
    results = fair_classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    
    CM2 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    return compare_fairness_metrics_as_df(CM1, CM2, "preprocessing")
        
    '''_________________________
    RW = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups)

    fair_dataset_train = RW.fit_transform(dataset_train)

    fair_RF = RandomForestClassifier(n_estimators=1100)
    fair_RF.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())

    results = fair_RF.predict(dataset_test.features)

    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM2 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    compare_fairness_metrics(CM1, CM2, side_by_side=True)'''

In [36]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [37]:
RF = RandomForestClassifier(n_estimators=1100)
RF.fit(dataset_train.features, dataset_train.labels.ravel())

results = RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM1 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)

RW = preprocessing.LFR(unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

fair_dataset_train = RW.fit_transform(dataset_train)

fair_RF = RandomForestClassifier(n_estimators=1100)
fair_RF.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())

results = fair_RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM2 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)